In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import os
from os import listdir
from os.path import isfile, join
import nibabel as nib
import imageio
import numpy as np
import torch
from torchvision.utils import save_image
import torchvision.transforms.v2.functional as TF
from torchvision.transforms import v2

import matplotlib.pyplot as plt
from PIL import Image

import nibabel as nib

In [ ]:
torch.manual_seed(13)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
data_dir = ("U-Net/data/NIFTIs_03/040102N")

In [ ]:
data_files = [f for f in sorted(listdir(data_dir)) if isfile(join(data_dir, f))]

# call train/test split based on len(data_files)
train_files = data_files[:4]
test_files = data_files[4:]

In [ ]:
data_files

Check both training files have same length

In [ ]:
for idx in range(0, len(train_files), 2):
  train_img = nib.load(join(data_dir, data_files[idx])).get_fdata()
  train_mask = nib.load(join(data_dir, data_files[idx+1])).get_fdata()

  print(f"Image file {idx//2} has {train_img.shape[0]} number of images")

Image file 0 has 352 number of images
Image file 1 has 352 number of images


Helper function to name saved image files

In [ ]:
def pad_int(run_id, zfill=4):
  # pad id number to 4 digits
  return str(run_id).zfill(zfill)

def get_fname(fnumber, img_bool):
  fnum = pad_int(fnumber)
  return f"{fnum}_img.png" if img_bool else f"{fnum}_msk.png"

Automate reading in of .img and .msk for U-Net training.

.msk files had multiple colors around segmentation instance confusing the neural network. To improve training data quality we ensure monochromatic segmentations. Only two colors should be present in masking images: one for the background and one for the segmented instance

Train file 1: 168-225 (green/yellow)

Train file 2: 452-497, 526-574 (green/yellow)
(offset by 352)

Test file: 98-142, 172-227 (green/yellow)

In [ ]:
save_img_dir = "U-Net/data/train_monochromatic/"

# iterate over train_data
t_file_id = 0
for idx in range(0, len(train_files), 2):
  train_img = nib.load(join(data_dir, data_files[idx])).get_fdata()
  train_mask = nib.load(join(data_dir, data_files[idx+1])).get_fdata()

  # loop over first dimension of image, every timeslice will be a training image
  for timeslice in range(train_img.shape[0]):
    # assign img/mask to corresponding slice
    train_slice = train_img[timeslice, :, :]
    mask_slice = train_mask[timeslice, :, :]
    # assign segmentation area pixels in mask to monochromatic pixel value (in this instance to 12.0)
    mask_slice = np.where(mask_slice != 0.0, 1938.0, 0.0) # 1938.0

    # create appropriate image/mask filename
    img_fname = get_fname(t_file_id*train_img.shape[0]+timeslice, img_bool=True)
    mask_fname = get_fname(t_file_id*train_img.shape[0]+timeslice, img_bool=False)

    # save image/mask
    plt.imsave(join(save_img_dir, img_fname), train_slice)
    plt.imsave(join(save_img_dir, mask_fname), mask_slice)

  t_file_id += 1

Repeat above process for the testing set

In [ ]:
save_img_dir = "U-Net/data/test_monochromatic/"

test_img = nib.load(join(data_dir, "040102_19010811_Sag_CUBE_T1_FS_img.nii")).get_fdata()
test_mask = nib.load(join(data_dir, "040102_19010811_Sag_CUBE_T1_FS_msk.nii")).get_fdata()

for timeslice in range(test_img.shape[0]):
  test_slice = test_img[timeslice, :, :]
  mask_slice = test_mask[timeslice, :, :]
  mask_slice = np.where(mask_slice != 0.0, 1938.0, 0.0) # 1938.0

  img_fname = get_fname(timeslice, img_bool=True)
  mask_fname = get_fname(timeslice, img_bool=False)
  plt.imsave(join(save_img_dir, img_fname), test_slice)
  plt.imsave(join(save_img_dir, mask_fname), mask_slice)


Check length of each data folder to ensure all images were correctly saved

In [ ]:
len(listdir("U-Net/data/train"))

1408

In [ ]:
len(listdir("U-Net/data/test_monochromatic"))

672